In [20]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures

churn_df = pd.read_csv('../Data/train_set.csv')

# churn_df = churn_df[churn_df['cancellation_request'] == True]
# churn_df = churn_df.drop('cancellation_request',axis=1)

col_names = churn_df.columns.tolist()

for i, name in enumerate(list(churn_df.columns.values)):
        if churn_df[name].dtype in [int, float]:
            churn_df[name] = churn_df.values[:, i]
        else:
            encoder = LabelEncoder()
            churn_df[name] = encoder.fit_transform(
                [str(i) for i in churn_df.values[:, i]]) 
to_drop = ['id']
churn_df = churn_df.drop(to_drop,axis=1)

churn_result = churn_df['renewed']
y = churn_result.astype(int)
churn_df = churn_df.drop('renewed',axis=1)

churn_df['account_value'] = churn_df['account_value'].replace(np.nan, 0)
churn_df = churn_df.replace(np.nan, 100000)

to_drop = ['job_count_over_client_count', 'job_count_over_invoice_count', 'job_count_over_quote_count', 
           'client_count_over_job_count','client_count_over_invoice_count', 'client_count_over_quote_count',
          'invoice_count_over_job_count', 'invoice_count_over_client_count', 'invoice_count_over_quote_count',
          'quote_count_over_job_count', 'quote_count_over_client_count', 'quote_count_over_invoice_count']

print(any(churn_df.isnull()))
churn_df = churn_df.fillna(0)
print(any(churn_df.isnull()))
churn_df[:]

True
True


,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,accepted_added_user_count,...,time_sheet_entries_used,users_logged_in,synced_to_quickbooks,phone_number,cancellation_request,account_value,plan_code,annual_plan,year,month
0,32,1.0,1.0,7.0,1.0,2,16,5,3,1,...,1,0,1,1,1,0.0,5,0,2016,2
1,42,100000.0,100000.0,100000.0,100000.0,2,22,2,2,0,...,0,1,0,1,0,91.0,5,0,2016,4
2,318,160.0,163.0,164.0,178.0,2,11,5,2,2,...,0,1,1,1,1,39.0,3,0,2015,7
3,61,100000.0,61.0,61.0,100000.0,2,22,2,0,0,...,0,1,0,1,1,0.0,5,0,2016,8
4,60,36.0,36.0,44.0,44.0,2,22,2,10,6,...,1,0,1,0,1,0.0,3,0,2015,5
5,334,50.0,57.0,56.0,88.0,2,11,5,3,3,...,0,1,1,0,1,0.0,3,0,2015,6
6,152,34.0,34.0,8.0,100000.0,2,39,1,16,10,...,1,0,0,0,1,0.0,5,0,2016,4
7,62,31.0,31.0,100000.0,100000.0,2,18,3,2,1,...,0,0,1,1,1,57.0,3,0,2014,11
8,323,21.0,1.0,7.0,1.0,2,41,4,1,0,...,0,0,0,1,0,39.0,3,0,2014,10
9,408,349.0,339.0,225.0,100000.0,2,39,1,15,0,...,1,0,0,1,1,39.0,3,0,2015,6


Ways to Preprocess Data (Brainstorming):
1. Put country/plan_code in distribution of most to least common...something with the mode 
2. 

In [21]:
 pd.isnull(churn_df).any(1).nonzero()[0]

array([], dtype=int64)

In [22]:
#Convert data to np array 
X = churn_df.as_matrix()

def ok(mat):
    print(np.any(np.isnan(mat)) and not np.all(np.isfinite(mat)))

ok(X)

False


In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

print("Feature space holds %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(y))

Feature space holds 3043 observations and 57 features
Unique target labels: [0 1]


In [24]:
from sklearn.model_selection import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),shuffle=True)
    y_pred = y.copy()

    # Iterate through folds
    for i, (train_index, test_index) in enumerate(KFold(n_splits=5).split(churn_df)):
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
       
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [25]:
from sklearn.svm import SVC

def accuracy(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)

print("Support vector machines:")
print("%.3f" % accuracy(y, run_cv(X,y,SVC)))


Support vector machines:
0.872
